In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

In [2]:

class Hydro:
    N = 10
    a = 0.0
    b = 1.0
    dx = 0.1
    t = 0.0
    cfl = 0.5
    
    gamma=5/3 ############### CHANGED FOR ISENTROPIC: 5/3
    p_l=1.0
    rho_l=1.0
    v_l=1.0
    p_r=0.125
    rho_r=0.1
    v_r=0.0
    e_l=1.0
    e_r=1.0
    E_l=1.0
    E_r=1.0
    h_l=0
    h_r=0
    x = None
    U = None
    F = None
    ap=None
    am=None
    rho= None
    P=None
    V=None
    h=None
    theta=1.5
    U1 = None
    U2 = None
    
    rhoL=None
    VL=None
    PL=None
    rho=None
    V=None
    P=None
    rhoR=None
    VR=None
    PR=None
    rhoR2=None
    VR2=None
    PR2=None
    first=0
    runs=0
    
    
    def __init__(self, N, a, b, t, cfl,p_l,p_r,rho_l,rho_r,v_l,v_r):
    
        if N%2!=0:
            N+=1
        mid=int(N/2)
        self.N = N
        self.a = a
        self.b = b
        self.t = t
        self.dx = (b-a) / float(N)
        self.p_l=float(p_l)
        self.p_r=float(p_r)
        self.rho_l=float(rho_l)
        self.rho_r=float(rho_r)
        self.v_l=float(v_l)
        self.v_r=float(v_r)
        self.U = np.empty([3,N]) #U-vector
        self.F = np.empty([3,N]) #F-vector
        
        self.rhoL=np.empty(N-3)
        self.rho=np.empty(N-3)
        self.rhoR=np.empty(N-3)
        self.rhoR2=np.empty(N-3)
        
        self.VL=np.empty(N-3)
        self.V=np.empty(N-3)
        self.VR=np.empty(N-3)
        self.VR2=np.empty(N-3)
        
        self.PL=np.empty(N-3)
        self.P=np.empty(N-3)
        self.PR=np.empty(N-3)
        self.PR2=np.empty(N-3)
        ##################
        self.rhoL[:mid]=self.rho_l
        self.rhoL[mid:]=self.rho_r
        self.rho[:mid-1]=self.rho_l
        self.rho[mid-1:]=self.rho_r
        self.rhoR[:mid-2]=self.rho_l
        self.rhoR[mid-2:]=self.rho_r
        self.rhoR2[:mid-3]=self.rho_l
        self.rhoR2[mid-3:]=self.rho_r
        
        self.VL[:mid]=self.v_l
        self.VL[mid:]=self.v_r
        self.V[:mid-1]=self.v_l
        self.V[mid-1:]=self.v_r
        self.VR[:mid-2]=self.v_l
        self.VR[mid-2:]=self.v_r
        self.VR2[:mid-3]=self.v_l
        self.VR2[mid-3:]=self.v_r
        
        self.PL[:mid]=self.p_l
        self.PL[mid:]=self.p_r
        self.P[:mid-1]=self.p_l
        self.P[mid-1:]=self.p_r
        self.PR[:mid-2]=self.p_l
        self.PR[mid-2:]=self.p_r
        self.PR2[:mid-3]=self.p_l
        self.PR2[mid-3:]=self.p_r
        
        
        
        
        
        
        
        self.x = a + self.dx*(np.arange(N)+0.5)
        self.cfl = float(cfl)
        
        
        
        self.e_l=self.p_l/(self.rho_l*(self.gamma-1.0))
        self.e_r=self.p_r/(self.rho_r*(self.gamma-1.0))
        #self.E_l=self.rho_l*self.e_l+0.5*self.rho_l*(self.v_l**2)
        #self.E_r=self.rho_r*self.e_r+0.5*self.rho_r*(self.v_r**2)
        self.h_l=1+self.e_l+self.p_l/self.rho_l
        self.h_r=1+self.e_r+self.p_r/self.rho_r
        
        
        
        
        self.U[0][:mid]=self.rho_l*self.W(self.v_l)
        self.U[1][:mid]=self.rho_l*self.v_l*self.h_l*self.W(self.v_l)**2
        self.U[2][:mid]=self.rho_l*self.h_l*self.W(self.v_l)**2-self.p_l-self.rho_l*self.W(self.v_l)
        
        self.U[0][mid:]=self.rho_r*self.W(self.v_r)
        self.U[1][mid:]=self.rho_r*self.v_r*self.h_r*self.W(self.v_r)**2
        self.U[2][mid:]=self.rho_r*self.h_r*self.W(self.v_r)**2-self.p_r-self.rho_r*self.W(self.v_r)
        
        self.F[0][:mid]=self.U[0][:mid]*self.v_l
        self.F[1][:mid]=self.U[1][:mid]*self.v_l+self.p_l
        self.F[2][:mid]=self.U[1][:mid]-self.U[0][:mid]*self.v_l
        
        self.F[0][mid:]=self.U[0][mid:]*self.v_r
        self.F[1][mid:]=self.U[1][mid:]*self.v_r+self.p_r
        self.F[2][mid:]=self.U[1][mid:]-self.U[0][mid:]*self.v_r
        
        
        #################Isentropic wave
        #x_0=0.5
        #sigma=0.4
        #rho_0=1
        #P_0=0.6
        #alpha=0.2
        #for i in range(0,N):
        #    x=a+i*self.dx
        #    fx=0
        #    if abs(x-x_0)<sigma:
        #        fx=(1-((x-x_0)/sigma)**2)**2
        #    rho=rho_0*(1+alpha*fx)
        #    P=P_0*(rho/rho_0)**self.gamma
        #    cs=(self.gamma*P/rho)**(1/2)
        #    cs0=(self.gamma*P_0/rho_0)**(1/2)
        #    v=(2/(self.gamma-1))*(cs-cs0)
        #    self.U[0][i]=rho
        #    self.U[1][i]=rho*v
        #    self.U[2][i]=(P/(self.gamma-1))+0.5*rho*v**2
            
        
        
        self.U1=np.copy(self.U)
        self.U2=np.copy(self.U)
        
        #CHANGE NEEDED HERE ALSO
        #self.rho=self.U[0]
        #self.V=self.U[1]/self.U[0]
        #self.P=(self.gamma-1)*(self.U[2]-0.5*(self.U[1]**2)/self.U[0])
        self.runs=0
    
    def evolve(self, tfinal):
        
        while self.t < tfinal:
           
            #Calculate fluxes
            udot = self.Lu(0)
            
            dt = self.getDt()
            if self.t + dt > tfinal:
                dt = tfinal - self.t
                
            left=2
            right=-2
            
            #update u
            self.U1[:,left:right]=self.U[:,left:right]+dt*udot[:,left:right]
            self.U2[:,left:right]=(3/4)*self.U[:,left:right]+(1/4)*self.U1[:,left:right]+(1/4)*dt*self.Lu(1)[:,left:right]
            self.U[:,left:right]=(1/3)*self.U[:,left:right]+(2/3)*self.U2[:,left:right]+(2/3)*dt*self.Lu(2)[:,left:right]
            
            
            #########UPDATE THESE    
            #Fluxes, not ignoring boundaries:
            #self.F[0]=np.copy(self.U[0]*self)
            #self.F[1]=np.copy(((self.U[1]**2)/self.U[0])\
            #                + (self.gamma-1.0)*(self.U[2]-0.5*((self.U[1]**2)/self.U[0])))
            #self.F[2]=np.copy((self.U[1]/self.U[0])*(self.U[2]
            #                +(self.gamma-1.0)*(self.U[2]-0.5*((self.U[1]**2)/self.U[0]))))
            self.t += dt
            
            ###########AND THESE
            #Update Quantity-Arrays:
            #self.rho=self.U[0]
            #self.V=self.U[1]/self.U[0]
            #self.P=(self.gamma-1)*(self.U[2]-0.5*(self.U[1]**2)/self.U[0])
            
            #for animation:
        #line.set_data(self.x[1:-1] , self.U[0][1:-1])
        
        #return line,
        
            
    def getDt(self):
        #rand=self.Lu()
        eigen=np.append(self.ap,self.am)
        thingy=np.fabs(eigen).max()
        if thingy==0:
            thingy=1
        return self.cfl * self.dx / thingy
    
    def W(self,v):
        return (1 / ((1-v**2)**(1/2)))
    
    def h(self,P,rho):
        return (1 + (P*self.gamma)/(rho*(self.gamma-1)))
    
    def getV(self,D,S,T,v):
        #def f(v,D,S,T):
        #    return ((self.gamma*v*(T-S*v+D)-S*(1-v**2))**2 - (v**2)*(1-v**2)*(D**2)*(self.gamma-1)**2)
        f=((self.gamma*v*(T-S*v+D)-S*(1-v**2))**2 - (v**2)*(1-v**2)*(D**2)*(self.gamma-1)**2)
        #def fprime(v,s):
        #    return (((v**2)*s)-1)
        for i in range(50):
            if abs(f)<1e-14:
                break
            fprime=2*(self.gamma*v*(T-S*v+D)-S*(1-v**2))*(self.gamma*T-2*self.gamma*S*v+self.gamma*D+2*S*v)-(2*v-4*v**3)*(D**2)*(self.gamma-1)**2
            vp=v-f/fprime
            v=vp
            f=((self.gamma*v*(T-S*v+D)-S*(1-v**2))**2 - (v**2)*(1-v**2)*(D**2)*(self.gamma-1)**2)
            
        if v>=1:
            v=0.99
        if v<0:
            v=(-v)
        
        return v
    
    def Lu(self, number):
        if number==1:
            U_used=np.copy(self.U1)
        elif number==2:
            U_used=np.copy(self.U2)
        else:
            U_used=np.copy(self.U)
        
        ap = np.empty(self.N-3)
        am = np.empty(self.N-3)
        #Temporary observable-storage, these get changed by the higher order scheme. One for left, one for right
        UTempL=np.empty([3,self.N-3])
        UTempR=np.empty([3,self.N-3])
        FTempL=np.empty([3,self.N-3])
        FTempR=np.empty([3,self.N-3])
        
        
        ############HERE
        VLcop=np.copy(self.VL)
        Vcop=np.copy(self.V)
        VRcop=np.copy(self.VR)
        VR2cop=np.copy(self.VR2)
        PLcop=np.copy(self.PL)
        Pcop=np.copy(self.P)
        PRcop=np.copy(self.PR)
        PR2cop=np.copy(self.PR2)
        rhoLcop=np.copy(self.rhoL)
        rhocop=np.copy(self.rho)
        rhoRcop=np.copy(self.rhoR)
        rhoR2cop=np.copy(self.rhoR2)
        for i in range(self.N-3):
            hL=self.h(self.PL[i],self.rhoL[i])
            VLcop[i]=self.getV(U_used[0][i],U_used[1][i],U_used[2][i],self.VL[i])
            rhoLcop[i]=U_used[0][i]/self.W(VLcop[i])
            PLcop[i]=(U_used[2][i]+rhoLcop[i]*self.W(VLcop[i])*(1-self.W(VLcop[i]))) / (((self.gamma*self.W(VLcop[i])**2)/(self.gamma-1))-1)
            if np.isnan(PLcop[i]):
                print("woops!")
                print(self.runs)
            if PLcop[i]<0:
                PLcop[i]=0
            h=self.h(self.P[i],self.rho[i])
            Vcop[i]=self.getV(U_used[0][i+1],U_used[1][i+1],U_used[2][i+1],self.V[i])
            rhocop[i]=U_used[0][i+1]/self.W(Vcop[i])
            Pcop[i]=(U_used[2][i+1]+rhocop[i]*self.W(Vcop[i])*(1-self.W(Vcop[i]))) / (((self.gamma*self.W(Vcop[i])**2)/(self.gamma-1))-1)
            if Pcop[i]<0:
                Pcop[i]=0
            hR=self.h(self.PR[i],self.rhoR[i])
            VRcop[i]=self.getV(U_used[0][i+2],U_used[1][i+2],U_used[2][i+2],self.VR[i])
            rhoRcop[i]=U_used[0][i+2]/self.W(VRcop[i])
            PRcop[i]=(U_used[2][i+2]+rhoRcop[i]*self.W(VRcop[i])*(1-self.W(VRcop[i]))) / (((self.gamma*self.W(VRcop[i])**2)/(self.gamma-1))-1)
            if PRcop[i]<0:
                PRcop[i]=0
            hR2=self.h(self.PR2[i],self.rhoR2[i])
            VR2cop[i]=self.getV(U_used[0][i+3],U_used[1][i+3],U_used[2][i+3],self.VR2[i])
            rhoR2cop[i]=U_used[0][i+3]/self.W(VR2cop[i])
            PR2cop[i]=(U_used[2][i+3]+rhoR2cop[i]*self.W(VR2cop[i])*(1-self.W(VR2cop[i]))) / (((self.gamma*self.W(VR2cop[i])**2)/(self.gamma-1))-1)
            if PR2cop[i]<0:
                PR2cop[i]=0
        self.rhoL=rhoLcop
        self.rho=rhocop
        self.rhoR=rhoRcop
        self.rhoR2=rhoR2cop
        
        self.VL=VLcop
        self.V=VLcop
        self.VR=VLcop
        self.VR2=VLcop
        
        self.PL=PLcop
        self.P=Pcop
        self.PR=PRcop
        self.PR2=PR2cop
        
        VL=np.copy(self.VL)
        V=np.copy(self.V)
        VR=np.copy(self.VR)
        VR2=np.copy(self.VR2)
        PL=np.copy(self.PL)
        P=np.copy(self.P)
        PR=np.copy(self.PR)
        PR2=np.copy(self.PR2)
        rhoL=np.copy(self.rhoL)
        rho=np.copy(self.rho)
        rhoR=np.copy(self.rhoR)
        rhoR2=np.copy(self.rhoR2)
        
        
        #rhoL=np.copy(U_used[0][:-3])
        #VL=np.copy(U_used[1][:-3]/U_used[0][:-3])
        #PL=np.copy((self.gamma-1)*(U_used[2][:-3]-0.5*(U_used[1][:-3]**2)/U_used[0][:-3])) 
        
        #rho=np.copy(U_used[0][1:-2])
        #V=np.copy(U_used[1][1:-2]/U_used[0][1:-2])
        #P=np.copy((self.gamma-1)*(U_used[2][1:-2]-0.5*(U_used[1][1:-2]**2)/U_used[0][1:-2]) )
        
        #rhoR=np.copy(U_used[0][2:-1])
        #VR=np.copy(U_used[1][2:-1]/U_used[0][2:-1])
        #PR=np.copy((self.gamma-1)*(U_used[2][2:-1]-0.5*(U_used[1][2:-1]**2)/U_used[0][2:-1]) )
        
        #rhoR2=np.copy(U_used[0][3:])
        #VR2=np.copy(U_used[1][3:]/U_used[0][3:])
        #PR2=np.copy((self.gamma-1)*(U_used[2][3:]-0.5*(U_used[1][3:]**2)/U_used[0][3:]) )
        
        ObservablesL=[rhoL,VL,PL]
        Observables=[rho,V,P]
        ObservablesR=[rhoR,VR,PR]
        ObservablesR2=[rhoR2,VR2,PR2]
        rhoTemp=np.copy(rho)
        VTemp=np.copy(V)
        PTemp=np.copy(P)
        rhoTempR=np.copy(rhoR)
        VTempR=np.copy(VR)
        PTempR=np.copy(PR)
        TempObs=[rhoTemp,VTemp,PTemp]
        TempObsR=[rhoTempR,VTempR,PTempR]
        #Change them!
        for obs in range(3):
            if obs==1:
                x=self.theta*(Observables[obs][:]-ObservablesL[obs][:])/(1-Observables[obs][:]*ObservablesL[obs][:])
                y=0.5*(ObservablesR[obs][:]-ObservablesL[obs][:])/(1-ObservablesR[obs][:]*ObservablesL[obs][:])
                z=self.theta*(ObservablesR[obs][:]-Observables[obs][:])/(1-ObservablesR[obs][:]*Observables[obs][:])
                mats=0.5*self.minmod(x,y,z)[:]
                TempObs[obs]=(TempObs[obs]+mats)/(1+TempObs[obs]*mats)
            else:
                x=self.theta*(Observables[obs][:]-ObservablesL[obs][:])
                y=0.5*(ObservablesR[obs][:]-ObservablesL[obs][:])
                z=self.theta*(ObservablesR[obs][:]-Observables[obs][:])
                TempObs[obs]=TempObs[obs]+0.5*self.minmod(x,y,z)[:]
        for obs in range(3):
            if obs==1:
                x=self.theta*(ObservablesR[obs][:]-Observables[obs][:])/(1-ObservablesR[obs][:]*Observables[obs][:])
                y=0.5*(ObservablesR2[obs][:]-Observables[obs][:])/(1-ObservablesR2[obs][:]*Observables[obs][:])
                z=self.theta*(ObservablesR2[obs][:]-ObservablesR[obs][:])/(1-ObservablesR2[obs][:]*ObservablesR[obs][:])
                mats=0.5*self.minmod(x,y,z)[:]
                TempObsR[obs]=(TempObsR[obs]-mats)/(1-TempObsR[obs]*mats)
            else:
                
                x=self.theta*(ObservablesR[obs][:]-Observables[obs][:])
                y=0.5*(ObservablesR2[obs][:]-Observables[obs][:])
                z=self.theta*(ObservablesR2[obs][:]-ObservablesR[obs][:])
                TempObsR[obs]=TempObsR[obs]-0.5*self.minmod(x,y,z)[:]
        rho=rhoTemp
        P=PTemp
        V=VTemp
        rhoR=rhoTempR
        PR=PTempR
        VR=VTempR
        
        ###########HERE
        UTempL[0]=rho*self.W(V)
        UTempL[1]=rho*V*self.h(P,rho)*self.W(V)**2
        UTempL[2]=rho*self.h(P,rho)*self.W(V)**2 - P-rho*self.W(V)
        UTempR[0]=rhoR*self.W(VR)
        UTempR[1]=rhoR*VR*self.h(PR,rhoR)*self.W(VR)**2
        UTempR[2]=rhoR*self.h(PR,rhoR)*self.W(VR)**2 - PR-rhoR*self.W(VR)
        FTempL[0]=UTempL[0]*V
        FTempL[1]=UTempL[1]*V+P
        FTempL[2]=UTempL[1]-UTempL[0]*V
        FTempR[0]=UTempR[0]*VR
        FTempR[1]=UTempR[1]*VR+PR
        FTempR[2]=UTempR[1]-UTempR[0]*VR
        UL=UTempL[:,:]
        UR=UTempR[:,:]
        FL=FTempL[:,:]
        FR=FTempR[:,:]
        
        
        
        
        csl=(self.gamma*P/(rho*self.h(P,rho)))**(1/2)
        csr=(self.gamma*PR/(rhoR*self.h(PR,rhoR)))**(1/2)
        
       
        
        
        lpl=(V+csl)/(1+V*csl)
        lpr=(VR+csr)/(1+VR*csr)
        lml=(V-csl)/(1-V*csl)
        lmr=(VR-csr)/(1-VR*csr)
        
            
                
        
        for i in range(self.N-3):
            ap[i] = max(0, lpl[i], lpr[i])
            am[i] = max(0,-lml[i],-lmr[i])
        self.ap=ap
        self.am=am
        
        #apam=np.add(ap,am)
        FHLL = (ap*FL + am*FR - ap*am*(UR-UL)) / (ap+am)
        LU = np.empty([3,self.N])
        LU[:,2:-2] = -(FHLL[:,1:] - FHLL[:,:-1]) / self.dx
        
        self.runs+=1
        return LU
    
    def minmod(self,x,y,z):
        sign_x=np.empty(len(x))
        sign_y=np.empty(len(x))
        sign_z=np.empty(len(x))
        minmod=np.empty(len(x))
        for i in range(len(x)):
            sign_x[i]=1
            sign_y[i]=1
            sign_z[i]=1
            if x[i]<0:
                sign_x[i]=-1
            if y[i]<0:
                sign_y[i]=-1
            if z[i]<0:
                sign_z[i]=-1
            minmod[i]=(1/4)*(abs(sign_x[i]+sign_y[i]))*(sign_x[i]+sign_z[i])*min(abs(x[i]),abs(y[i]),abs(z[i]))
        return minmod
    
    def plot(self, ax=None, filename=None):
        
        if ax is None:
            fig, ax = plt.subplots(3, sharex=True, figsize=(5,10))
        else:
            fig = ax.get_figure()
        
        ax[0].set_title("t = "+str(self.t), fontsize=24)
        ax[0].plot(self.x[:-3], self.rho, 'k-')
        ax[0].set_ylabel('rho', fontsize=16)
        #ax[1].plot(self.x, self.V, 'k-')
        #ax[1].set_ylabel('v', fontsize=16)
        #ax[2].plot(self.x, self.P, 'k-')
        #ax[2].set_ylabel('p', fontsize=16)
        #ax[2].set_xlabel('x', fontsize=16)
        plt.tight_layout()
        #if filename is not None:
         #   fig.savefig(filename)
            
        return ax

In [3]:
#(self, N, a, b, t, cfl,p_l,p_r,rho_l,rho_r,v_l,v_r)
test=Hydro(100,0,2,0,0.1,1.0,0.125,1.0,0.1,0.0,0.0)
#ax=test.densplot()
test.evolve(1)
#ay=test.densplot()
test.plot()
plt.show()

/home/mats/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:232: RuntimeWarning: invalid value encountered in double_scalars


woops!
632
woops!
632
woops!
632
woops!
633
woops!
633
woops!
633
woops!
633
woops!
633
woops!
633
woops!
634
woops!
634
woops!
634
woops!
634
woops!
634
woops!
634
woops!
634
woops!
634
woops!
634
woops!
635
woops!
635
woops!
635
woops!
635
woops!
635
woops!
635
woops!
635
woops!
635
woops!
635
woops!
635
woops!
635
woops!
635
woops!
636
woops!
636
woops!
636
woops!
636
woops!
636
woops!
636
woops!
636
woops!
636
woops!
636
woops!
636
woops!
636
woops!
636
woops!
636
woops!
636
woops!
636
woops!
637
woops!
637
woops!
637
woops!
637
woops!
637
woops!
637
woops!
637
woops!
637
woops!
637
woops!
637
woops!
637
woops!
637
woops!
637
woops!
637
woops!
637
woops!
637
woops!
637
woops!
637
woops!
638
woops!
638
woops!
638
woops!
638
woops!
638
woops!
638
woops!
638
woops!
638
woops!
638
woops!
638
woops!
638
woops!
638
woops!
638
woops!
638
woops!
638
woops!
638
woops!
638
woops!
638
woops!
638
woops!
638
woops!
638
woops!
639
woops!
639
woops!
639
woops!
639
woops!
639
woops!
639
woops!
639

KeyboardInterrupt: 

In [15]:

test.U[2]

array([ 3.   ,  3.   ,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
          nan,    nan,  0.375,  0.375])